# **Detecção de Fraudes com Keras**

**Objetivo**

Treinar um modelo de Redes Neurais para detecção de fraudes em transações financeiras.

### **1. Introdução**

### **2. Biblioteca e Base de Dados**

In [1]:
# Carregando biblioteca

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
# Carregando base de dados

dados = pd.read_csv('/content/drive/MyDrive/Projetos/Detecção de Fraudes/Dados/creditcard.csv')

In [3]:
# Visualizando base de dados

dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Visualizando informações sobre os dados

dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
# Verificar distribuição das classes

print("Distribuição das classes:")
print(dados['Class'].value_counts())

Distribuição das classes:
0    284315
1       492
Name: Class, dtype: int64


### **3. Pré-Processamento de Dados**

In [6]:
# Separando os dados de entrada (features) e as classes

features = dados.drop('Class', axis=1)
classes = dados['Class']

In [7]:
# Padronizando as features

# Instânciando objeto
scaler = StandardScaler()

# Aplicando aos dados
features['Time'] = scaler.fit_transform(features['Time'].values.reshape(-1, 1))
features['Amount'] = scaler.fit_transform(features['Amount'].values.reshape(-1, 1))

In [8]:
# Undersampling: balanceando os dados

# Instânciando objeto
undersampler = RandomUnderSampler(sampling_strategy = 0.5, random_state = 42)

# Aplicando aos dados
features_resampled, classes_resampled = undersampler.fit_resample(features,
                                                                  classes)

In [9]:
# Dividindo os dados em treino e teste

X_treino, X_teste, y_treino, y_teste = train_test_split(features_resampled,
                                                        classes_resampled,
                                                        test_size = 0.3,
                                                        random_state = 42)

### **4. Construção e Treinamento do Modelo de Redes Neurais**

In [10]:
# Criar o modelo da rede neural

modelo = Sequential()
modelo.add(Dense(64, activation = 'relu', input_dim = X_treino.shape[1]))
modelo.add(Dropout(0.2))
modelo.add(Dense(32, activation = 'relu'))
modelo.add(Dropout(0.2))
modelo.add(Dense(1, activation = 'sigmoid'))

In [11]:
# Compilar o modelo

modelo.compile(loss = 'binary_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [12]:
# Treinando o modelo

modelo.fit(X_treino, y_treino, epochs = 20, batch_size = 32, verbose = 1)

Epoch 1/20
33/33 [==============================] - 3s 5ms/step - loss: 0.5997 - accuracy: 0.6447
Epoch 2/20
33/33 [==============================] - 0s 8ms/step - loss: 0.3493 - accuracy: 0.8877
Epoch 3/20
33/33 [==============================] - 0s 7ms/step - loss: 0.2530 - accuracy: 0.9313
Epoch 4/20
33/33 [==============================] - 0s 6ms/step - loss: 0.2007 - accuracy: 0.9361
Epoch 5/20
33/33 [==============================] - 0s 5ms/step - loss: 0.1745 - accuracy: 0.9448
Epoch 6/20
33/33 [==============================] - 0s 5ms/step - loss: 0.1481 - accuracy: 0.9506
Epoch 7/20
33/33 [==============================] - 0s 7ms/step - loss: 0.1468 - accuracy: 0.9487
Epoch 8/20
33/33 [==============================] - 0s 6ms/step - loss: 0.1351 - accuracy: 0.9526
Epoch 9/20
33/33 [==============================] - 0s 8ms/step - loss: 0.1224 - accuracy: 0.9584
Epoch 10/20
33/33 [==============================] - 0s 6ms/step - loss: 0.1241 - accuracy: 0.9564
Epoch 11/20
33/33 [

In [13]:
# Aplicando o modelo no conjunto de teste

previsoes_proba = modelo.predict(X_teste)
previsoes = np.where(previsoes_proba >= 0.5, 1, 0)

14/14 [==============================] - 1s 3ms/step


### **5. Avaliação do Modelo**

In [14]:
# Métricas do Modelo

print(classification_report(y_teste, previsoes))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       311
           1       0.95      0.92      0.93       132

    accuracy                           0.96       443
   macro avg       0.96      0.95      0.95       443
weighted avg       0.96      0.96      0.96       443

